In [4]:
import pandas as pd
import numpy as np
import pickle
from sklearn.model_selection import train_test_split, cross_val_score

In [5]:
df = pd.read_csv("./laptop_price.csv", encoding="latin-1")
df.head()

,laptop_ID,Company,Product,TypeName,Inches,ScreenResolution,Cpu,Ram,Gpu,OpSys,Weight,Price_euros
0,1,Apple,MacBook Pro,Ultrabook,13.3,IPS Panel Retina Display 2560x1600,Intel Core i5 2.3GHz,8GB,Intel Iris Plus Graphics 640,macOS,1.37kg,1339.69
1,2,Apple,Macbook Air,Ultrabook,13.3,1440x900,Intel Core i5 1.8GHz,8GB,Intel HD Graphics 6000,macOS,1.34kg,898.94
2,3,HP,250 G6,Notebook,15.6,Full HD 1920x1080,Intel Core i5 7200U 2.5GHz,8GB,Intel HD Graphics 620,No OS,1.86kg,575.00
3,4,Apple,MacBook Pro,Ultrabook,15.4,IPS Panel Retina Display 2880x1800,Intel Core i7 2.7GHz,16GB,AMD Radeon Pro 455,macOS,1.83kg,2537.45
4,5,Apple,MacBook Pro,Ultrabook,13.3,IPS Panel Retina Display 2560x1600,Intel Core i5 3.1GHz,8GB,Intel Iris Plus Graphics 650,macOS,1.37kg,1803.60


In [6]:
df.shape

(1303, 12)

In [7]:
df.isnull().sum()

laptop_ID           0
Company             0
Product             0
TypeName            0
Inches              0
ScreenResolution    0
Cpu                 0
Ram                 0
Gpu                 0
OpSys               0
Weight              0
Price_euros         0
dtype: int64

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1303 entries, 0 to 1302
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   laptop_ID         1303 non-null   int64  
 1   Company           1303 non-null   object 
 2   Product           1303 non-null   object 
 3   TypeName          1303 non-null   object 
 4   Inches            1303 non-null   float64
 5   ScreenResolution  1303 non-null   object 
 6   Cpu               1303 non-null   object 
 7   Ram               1303 non-null   object 
 8   Gpu               1303 non-null   object 
 9   OpSys             1303 non-null   object 
 10  Weight            1303 non-null   object 
 11  Price_euros       1303 non-null   float64
dtypes: float64(2), int64(1), object(9)
memory usage: 122.3+ KB


In [9]:
df["Ram"] = df["Ram"].str.replace("GB", "").astype('int32')
df["Weight"] = df["Weight"].str.replace("kg", "").astype('float32')

In [10]:
df.corr(numeric_only=True)["Price_euros"]

laptop_ID      0.067830
Inches         0.068197
Ram            0.743007
Weight         0.210370
Price_euros    1.000000
Name: Price_euros, dtype: float64

In [11]:
def change_company(name):
    if name in ("Samsung", "Razer", "Mediacom", "Microsoft", "Xiaomi", "Vero", "Chuwi", "Google", "Fujitsu", "LG", "Huawei"):
        return "Other"
    return name
df["Company"] = df["Company"].apply(change_company)

In [12]:
df.head()

,laptop_ID,Company,Product,TypeName,Inches,ScreenResolution,Cpu,Ram,Gpu,OpSys,Weight,Price_euros
0,1,Apple,MacBook Pro,Ultrabook,13.3,IPS Panel Retina Display 2560x1600,Intel Core i5 2.3GHz,8,Intel Iris Plus Graphics 640,macOS,1.37,1339.69
1,2,Apple,Macbook Air,Ultrabook,13.3,1440x900,Intel Core i5 1.8GHz,8,Intel HD Graphics 6000,macOS,1.34,898.94
2,3,HP,250 G6,Notebook,15.6,Full HD 1920x1080,Intel Core i5 7200U 2.5GHz,8,Intel HD Graphics 620,No OS,1.86,575.00
3,4,Apple,MacBook Pro,Ultrabook,15.4,IPS Panel Retina Display 2880x1800,Intel Core i7 2.7GHz,16,AMD Radeon Pro 455,macOS,1.83,2537.45
4,5,Apple,MacBook Pro,Ultrabook,13.3,IPS Panel Retina Display 2560x1600,Intel Core i5 3.1GHz,8,Intel Iris Plus Graphics 650,macOS,1.37,1803.60


In [13]:
df["TypeName"].value_counts()

TypeName
Notebook              727
Gaming                205
Ultrabook             196
2 in 1 Convertible    121
Workstation            29
Netbook                25
Name: count, dtype: int64

In [14]:
df["Touch"] = df["ScreenResolution"].apply(lambda x: 1 if "Touchscreen" in x else 0)
df["IPS"] = df["ScreenResolution"].apply(lambda x: 1 if "IPS" in x else 0)

In [15]:
df.head()

,laptop_ID,Company,Product,TypeName,Inches,ScreenResolution,Cpu,Ram,Gpu,OpSys,Weight,Price_euros,Touch,IPS
0,1,Apple,MacBook Pro,Ultrabook,13.3,IPS Panel Retina Display 2560x1600,Intel Core i5 2.3GHz,8,Intel Iris Plus Graphics 640,macOS,1.37,1339.69,0,1
1,2,Apple,Macbook Air,Ultrabook,13.3,1440x900,Intel Core i5 1.8GHz,8,Intel HD Graphics 6000,macOS,1.34,898.94,0,0
2,3,HP,250 G6,Notebook,15.6,Full HD 1920x1080,Intel Core i5 7200U 2.5GHz,8,Intel HD Graphics 620,No OS,1.86,575.00,0,0
3,4,Apple,MacBook Pro,Ultrabook,15.4,IPS Panel Retina Display 2880x1800,Intel Core i7 2.7GHz,16,AMD Radeon Pro 455,macOS,1.83,2537.45,0,1
4,5,Apple,MacBook Pro,Ultrabook,13.3,IPS Panel Retina Display 2560x1600,Intel Core i5 3.1GHz,8,Intel Iris Plus Graphics 650,macOS,1.37,1803.60,0,1


In [16]:
df["Cpu"] = df["Cpu"].apply(lambda x: " ".join(x.split()[:3]))

In [17]:
def change_cpu(name):
    if "Intel Core i" in name:
        return name
    if name.split()[0] == "AMD":
        return "AMD"
    return "Other"
df["Processor"] = df["Cpu"].apply(change_cpu)

In [18]:
df["Gpu"].value_counts()

Gpu
Intel HD Graphics 620      281
Intel HD Graphics 520      185
Intel UHD Graphics 620      68
Nvidia GeForce GTX 1050     66
Nvidia GeForce GTX 1060     48
                          ... 
Nvidia Quadro M500M          1
AMD Radeon R7 M360           1
Nvidia Quadro M3000M         1
Nvidia GeForce 960M          1
ARM Mali T860 MP4            1
Name: count, Length: 110, dtype: int64

In [19]:
def change_gpu(name):
    if "GTX" in name:
        return "Nvidia GTX"
    if "RTX" in name:
        return "Nvidia RTX"
    if "Nvidia" in name:
        return "Nvidia"
    if "AMD" in name:
        return "AMD"
    if "Intel" in name:
        return "Intel"
    return name
df["Gpu"] = df["Gpu"].apply(change_gpu)

In [20]:
df["Gpu"].value_counts()

Gpu
Intel                722
Nvidia GTX           239
AMD                  180
Nvidia               161
ARM Mali T860 MP4      1
Name: count, dtype: int64

In [21]:
df.rename(columns = {'Gpu':'GPU'}, inplace = True)
df.rename(columns = {'OpSys':'OS'}, inplace = True)

In [22]:
df = df[df["GPU"] != "ARM Mali T860 MP4"]

In [23]:
def change_os(name):
    if "Linux" in name:
        return "Linux"
    if "Windows" in name:
        return "Windows"
    if "mac" in name.lower() :
        return "Mac"
    return "Other"
df["OS"] = df["OS"].apply(change_os)

In [24]:
df = df.drop(columns=["laptop_ID", "Inches", "ScreenResolution", "Cpu", "Product"])

In [25]:
df.head()

,Company,TypeName,Ram,GPU,OS,Weight,Price_euros,Touch,IPS,Processor
0,Apple,Ultrabook,8,Intel,Mac,1.37,1339.69,0,1,Intel Core i5
1,Apple,Ultrabook,8,Intel,Mac,1.34,898.94,0,0,Intel Core i5
2,HP,Notebook,8,Intel,Other,1.86,575.00,0,0,Intel Core i5
3,Apple,Ultrabook,16,AMD,Mac,1.83,2537.45,0,1,Intel Core i7
4,Apple,Ultrabook,8,Intel,Mac,1.37,1803.60,0,1,Intel Core i5


In [26]:
df["Company"].value_counts()

Company
Dell       297
Lenovo     297
HP         274
Asus       158
Acer       103
MSI         54
Other       50
Toshiba     48
Apple       21
Name: count, dtype: int64

In [27]:
df.shape

(1302, 10)

In [28]:
df = pd.get_dummies(df)

In [29]:
df.head()

,Ram,Weight,Price_euros,Touch,IPS,Company_Acer,Company_Apple,Company_Asus,Company_Dell,Company_HP,...,GPU_Nvidia GTX,OS_Linux,OS_Mac,OS_Other,OS_Windows,Processor_AMD,Processor_Intel Core i3,Processor_Intel Core i5,Processor_Intel Core i7,Processor_Other
0,8,1.37,1339.69,0,1,False,True,False,False,False,...,False,False,True,False,False,False,False,True,False,False
1,8,1.34,898.94,0,0,False,True,False,False,False,...,False,False,True,False,False,False,False,True,False,False
2,8,1.86,575.00,0,0,False,False,False,False,True,...,False,False,False,True,False,False,False,True,False,False
3,16,1.83,2537.45,0,1,False,True,False,False,False,...,False,False,True,False,False,False,False,False,True,False
4,8,1.37,1803.60,0,1,False,True,False,False,False,...,False,False,True,False,False,False,False,True,False,False


In [30]:
x = df.drop("Price_euros", axis=1)
y = df["Price_euros"]

In [31]:
x_train, x_test, y_train, y_test = train_test_split(x, y)

In [32]:
def get_model_accuracy(model):
    model.fit(x_train, y_train)
    accuracy = model.score(x_test, y_test)
    print(f'{str(model)} Accuracy --> {str(accuracy)}')

In [33]:
from sklearn.linear_model import LinearRegression
lr_model = LinearRegression()
cross_val_score(lr_model, x, y, cv=5).mean()
# get_model_accuracy(lr_model)

np.float64(0.6824002220099864)

In [34]:
from sklearn.linear_model import Lasso
lasso_model = Lasso()
cross_val_score(lasso_model, x, y, cv=5).mean()
# get_model_accuracy(lasso_model)

np.float64(0.6833918328068679)

In [35]:
from sklearn.tree import DecisionTreeRegressor
dt_model = DecisionTreeRegressor()
cross_val_score(dt_model, x, y, cv=5).mean()
# get_model_accuracy(dt_model)

np.float64(0.6212726143338317)

In [36]:
from sklearn.ensemble import RandomForestRegressor
rf_model = RandomForestRegressor()
# cross_val_score(rf_model, x, y, cv=5).mean()
get_model_accuracy(rf_model)

RandomForestRegressor() Accuracy --> 0.7245918643039418


In [37]:
from sklearn.model_selection import GridSearchCV
params = {
    'n_estimators': [10, 50, 100],
    'criterion': ["squared_error", "friedman_mse", "squared_error"]
}

grid = GridSearchCV(estimator=rf_model, param_grid=params)
grid_fit = grid.fit(x_train, y_train)
best = grid_fit.best_estimator_
best

c:\Users\hesha\.virtualenvs\laptop-price-predictor-CP8QwEqK\lib\site-packages\numpy\ma\core.py:2881: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,


RandomForestRegressor()

In [38]:
best.score(x_test, y_test)

0.7380571224424847

In [39]:
# with open("model.pickle", "wb") as file:
#     pickle.dump(best, file)